In [1]:
# initialize project:
using Pkg; Pkg.activate("./MyProj");

  Activating project at `~/GitHub/CNCSim/MyProj`


In [2]:
using JLD2
using JuMP
using GLPK
using LinearAlgebra

In [5]:
# include packages:
include("./libs/cnc.jl"); include("./libs/utils.jl");

# magic states

In [6]:
# Define magic state
n = 8; T = Vector{Real}([1,1/sqrt(2),1/sqrt(2),0]);

In [7]:
ps = [PauliString(n) for n in 1:4];

In [8]:
T_states = Vector{Vector{Real}}([T]); T_state = T;
for i in 1:(n-1)
    T_state = kron(T_state,T); push!(T_states,T_state);
end

In [9]:
T_states

8-element Vector{Vector{Real}}:
 [1.0, 0.7071067811865475, 0.7071067811865475, 0.0]
 [1.0, 0.7071067811865475, 0.7071067811865475, 0.0, 0.7071067811865475, 0.4999999999999999, 0.4999999999999999, 0.0, 0.7071067811865475, 0.4999999999999999, 0.4999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0]
 [1.0, 0.7071067811865475, 0.7071067811865475, 0.0, 0.7071067811865475, 0.4999999999999999, 0.4999999999999999, 0.0, 0.7071067811865475, 0.4999999999999999  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [1.0, 0.7071067811865475, 0.7071067811865475, 0.0, 0.7071067811865475, 0.4999999999999999, 0.4999999999999999, 0.0, 0.7071067811865475, 0.4999999999999999  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [1.0, 0.7071067811865475, 0.7071067811865475, 0.0, 0.7071067811865475, 0.4999999999999999, 0.4999999999999999, 0.0, 0.7071067811865475, 0.4999999999999999  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [1.0, 0.7071067811865475, 0.7071067811865475, 0.0, 0.7071067811865475, 0.4999999999

# stabs

In [14]:
cncs_n_0 = Vector{Matrix{Rational{Int64}}}([]);;
for n in 1:4
    cnc_sets_n_0 = generate_all_cncs(n,[0]);
    cnc_n_0 = generate_cnc_vertices(n,cnc_sets_n_0);
    push!(cncs_n_0,cnc_n_0);
end

In [33]:
for n in 1:4
    stab_n = cncs_n_0[n];
    @save "./data/stab_$(n).jld" stab_n
end

# cncs

In [9]:
cncs_n_1 = Vector{Matrix{Rational{Int64}}}([]);;
for n in 1:3
    cnc_sets_n_1 = generate_all_cncs(n,[1]);
    cnc_n_1 = generate_cnc_vertices(n,cnc_sets_n_1);
    push!(cncs_n_1,cnc_n_1);
end

In [10]:
cnc_4_1 = JLD2.load("./data/cnc_vectors_4_1.jld")["cnc_vectors_4_1"];
push!(cncs_n_1,cnc_4_1)

4-element Vector{Matrix{Rational{Int64}}}:
 [1 1 … 1 1; -1 1 … -1 -1; -1 -1 … 1 -1; 1 1 … -1 -1]
 [1 1 … 1 1; -1 0 … 1 0; … ; 0 1 … 0 1; 0 0 … 0 0]
 [1 1 … 1 1; 0 0 … 1 0; … ; 0 0 … 0 -1; 0 0 … 0 0]
 [1 1 … 1 1; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]

In [30]:
for n in 1:3
    cnc_n_1 = cncs_n_1[n];
    @save "./data/cnc_$(n)_1.jld" cnc_n_1
end

# phase space robustness

In [13]:
for n in 1:3
    # Create a model with GLPK as the solver
    model = Model(GLPK.Optimizer)
    N = size(cncs_n_1[n])[2];

    # Define the decision variables
    @variable(model, x[1:N])
    @variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

    # Objective: Minimize the sum of u (which represents |x|)
    @objective(model, Min, sum(u))

    # Constraints for the absolute values
    @constraint(model, [i=1:N], u[i] >= x[i])
    @constraint(model, [i=1:N], u[i] >= -x[i])

    # Equality constraint: M * x = b
    @constraint(model, Matrix{Rational{Int64}}(cncs_n_1[n]) * x .== T_states[n])

    # Solve the model
    optimize!(model)

    # Get the results
    x_value = value.(x)
    obj_value = objective_value(model)  # Use a different variable name

    # Print the results
    println("Optimal objective value (min ||x||_1): ", obj_value)
    println("Optimal value of x: ", x_value,"\n")

    @save "./data/cnc_decomposition_$(n).jld" x_value
end

Optimal objective value (min ||x||_1): 1.0
Optimal value of x: [0.14644660940672627, 0.0, 0.0, 0.5, 0.35355339059327373, 0.0, 0.0, 0.0]

Optimal objective value (min ||x||_1): 0.9999999999999998
Optimal value of x: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.060660171779821304, 0.0, -2.6538648049016755e-33, 3.9337420619235914e-17, 0.0, 0.0, 0.0, -2.385974376633503e-33, 1.9414134467048865e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.4694469519536134e-18, 0.0, 0.0, 9.85809497670568e-34, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04289321881345244, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14644660940672613, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3804259570458955e-33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1464466094067262, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.442293931007186e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -4.918504322276986e-18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

# robustness of magic

In [35]:
for n in 1:4
    # Create a model with GLPK as the solver
    model = Model(GLPK.Optimizer)
    N = size(cncs_n_0[n])[2];

    # Define the decision variables
    @variable(model, x[1:N])
    @variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

    # Objective: Minimize the sum of u (which represents |x|)
    @objective(model, Min, sum(u))

    # Constraints for the absolute values
    @constraint(model, [i=1:N], u[i] >= x[i])
    @constraint(model, [i=1:N], u[i] >= -x[i])

    # Equality constraint: M * x = b
    @constraint(model, Matrix{Rational{Int64}}(cncs_n_0[n]) * x .== T_states[n])

    # Solve the model
    optimize!(model)

    # Get the results
    x_value = value.(x)
    obj_value = objective_value(model)  # Use a different variable name

    # Print the results
    println("Optimal objective value (min ||x||_1): ", obj_value)
    println("Optimal value of x: ", x_value,"\n")

    @save "./data/stab_decomposition_$(n).jld" x_value
end

Optimal objective value (min ||x||_1): 1.414213562373095
Optimal value of x: [0.0, -0.20710678118654746, 0.0, 0.0, 0.7071067811865475, 0.5]

Optimal objective value (min ||x||_1): 1.7475468957064282
Optimal value of x: [0.35355339059327373, 0.0, 1.5419764230904951e-18, -3.854941057726237e-19, 1.927470528863119e-18, 0.0, 0.0, 0.0, 4.163336342344337e-17, -0.1666666666666666, 0.0, 0.0, 0.0, 0.0, 9.62964972193618e-35, -7.7098821154524765e-19, 0.0, 0.0, -3.854941057726236e-19, 0.0, 0.0, 0.3333333333333333, 1.5419764230904951e-18, 0.0, 0.0, 0.0, -1.5419764230904951e-18, -3.8549410577262387e-19, 0.35355339059327373, 0.0, -0.02022005725994047, 0.0, 0.0, 0.0, 0.0, 2.3129646346357427e-18, 0.0, 0.0, 0.0, 6.938893903907228e-17, 0.0, -0.16666666666666657, -3.8549410577262397e-19, 0.0, 0.3333333333333333, -0.020220057259940444, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.8549410577262387e-19, -7.709882115452476e-19, -1.1564823173178713e-18, 1.5419764230904951e-18, -3.0839528461809902e-18, 0.0]

Optim

# tensoring cnc with stabilizer

In [47]:
stab_4_decomposition = load("./data/stab_decomposition_4.jld")["x_value"]
s4 = cncs_n_0[4][:,findall(x->abs(x)>10^-14,stab_4_decomposition)]

256×86 Matrix{Rational{Int64}}:
 1  1   1  1  1  1  1  1  1  1  1  1  …   1  1  1  1  1   1  1   1  1   1  1
 0  0   0  0  1  0  0  0  1  1  1  0      0  1  1  0  1   0  0   0  0   0  1
 0  0   0  0  0  0  0  1  0  0  0  1      0  0  0  1  0   0  1   1  0   0  0
 0  0   0  0  0  0  0  0  0  0  0  0      0  0  0  0  0   0  0   0  0   0  0
 1  0   0  1  0  1  1  0  1  0  0  0      0  1  0  1  0   0  0   0  1   0  0
 0  0   0  0  0  0  0  0  1  0  0  0  …   0  1  0  0  0  -1  0   0  0  -1  0
 0  0  -1  0  0  0  0  0  0  0  0  0     -1  0  0  1  0   0  0   0  0   0  0
 0  0   0  0  0  0  0  0  0  0  0  0      0  0  0  0  0   0  0   0  0   0  0
 0  1   0  0  1  0  0  0  0  1  1  0      0  0  0  0  0   0  1   1  0   0  0
 0  0  -1  0  1  0  0  0  0  1  1  0     -1  0  0  0  0   0  0   0  0   0  0
 ⋮               ⋮              ⋮     ⋱   ⋮               ⋮                ⋮
 0  0   0  0  0  0  0  0  0  0  0  0      0  0  0  0  0   0  0   0  0   0  0
 0  0   0  0  0  0  0  0  0  0  0  0      0 

In [48]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(s4)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(s4) * x .== T_states[4])

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value,"\n")

Optimal objective value (min ||x||_1): 2.8627416997969517
Optimal value of x: [-0.011719426993575824, -0.011719426993576008, -0.03347086912079595, 0.0022970773009201185, 0.1372583002030469, -0.02573593128807163, -0.02343885398715187, -0.025735931288071445, 0.07781745930520259, 0.0915999231107188, 0.08700576850887974, 0.012867965644035646, -0.026884469938531316, -0.02688446993853122, 0.13725830020304736, -0.03461940777125581, 0.01401650429449549, -0.02573593128807158, -0.015469875665448697, -0.013172798364529843, 0.014016504294495942, 0.01401650429449563, -0.01317279836453051, 0.0008437059299664443, 0.08470869120796012, 0.012867965644035704, 0.1372583002030493, -0.0022970773009196007, -0.023438853987152466, 0.012867965644035723, 0.13725830020304844, 0.004289321881344371, -0.03262716319082982, -0.011719426993576372, 0.0008437059299661598, 0.004289321881345005, -0.012867965644035646, 0.13725830020304855, -0.03461940777125583, -0.026884469938531372, -0.008578643762690474, -0.01171942699357

In [52]:
cnc_3_decomposition = load("./data/cnc_decomposition_3.jld")["x_value"]
c3 = cncs_n_1[3][:,findall(x->abs(x)>10^-14,cnc_3_decomposition)]

64×36 Matrix{Rational{Int64}}:
 1  1  1  1   1  1  1   1   1  1   1  …   1  1   1  1  1   1  1  1   1  1  1
 0  1  0  1   1  0  0   0   0  0   1      0  1   1  1  1   0  0  0   0  1  1
 0  0  1  0   1  1  0   0   0  1   1      0  0   1  0  1   1  1  1   0  1  1
 0  0  0  0  -1  0  0   0   0  0  -1      0  0  -1  0  1   0  0  0   0  1  1
 0  1  0  1   0  1  0   0   1  1   1      0  1   0  1  1   0  1  1   0  1  0
 0  1  0  1   0  0  0   1   0  0   1  …   0  1   0  1  1   0  0  0   1  1  0
 1  0  0  0   0  1  1   0   0  1   1      1  0   0  0  1   0  1  1   0  1  0
 0  0  0  0   0  0  0   0   0  0  -1      0  0   0  0  1   0  0  0   0  1  0
 0  1  1  0   1  0  0   0   1  1   0      0  0   1  1  0   1  1  1   0  0  1
 1  1  0  0   1  0  0   0   0  0   0      0  0   1  1  0   0  0  0   0  0  1
 ⋮               ⋮                 ⋮  ⋱   ⋮                ⋮               ⋮
 0  0  0  0   0  0  0   0   1  0   0  …   0  0   0  0  0   0  0  0   0  0  0
 0  0  1  0   0  0  0   0   0  0   0      0  

In [ ]:
c7 = kron(c3,s4)

65536×18576 Matrix{Rational{Int64}}:
  1   1  1   1  1  1   1   1  1   1  1  …  1   1  1  1   1   1  1   1  1  1
 -1   0  0   0  1  0  -1   0  0   0  1     0   0  1  0  -1   0  0   0  1  0
  0  -1  0   0  0  1   0  -1  0   0  0     0   0  0  1   0  -1  0   0  0  1
  0   0  1  -1  0  0   0   0  1  -1  0     1  -1  0  0   0   0  1  -1  0  0
  0   0  0   0  0  0   0   0  0   0  0     0   0  0  0   1   1  1   1  1  1
  0   0  0   0  0  0   0   0  0   0  0  …  0   0  0  0  -1   0  0   0  1  0
  0   0  0   0  0  0   0   0  0   0  0     0   0  0  0   0  -1  0   0  0  1
  0   0  0   0  0  0   0   0  0   0  0     0   0  0  0   0   0  1  -1  0  0
  0   0  0   0  0  0   0   0  0   0  0     0   0  0  0   0   0  0   0  0  0
  0   0  0   0  0  0   0   0  0   0  0     0   0  0  0   0   0  0   0  0  0
  ⋮                ⋮                 ⋮  ⋱                ⋮                ⋮
  0   0  0   0  0  0   0   0  0   0  0     0   0  0  0   0   0  0   0  0  0
  0   0  0   0  0  0   0   0  0   0  0     0   0  0

In [62]:
@save "./data/cnc_stab_7.jld" c7

In [57]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c7)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c7) * x .== T_states[7])

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value,"\n")

Optimal objective value (min ||x||_1): 3.672447327281913
Optimal value of x: [-1.4723271908038689e-5, -1.4723271902432857e-5, -4.204989778647264e-5, 2.885848748652047e-6, 0.00017243942701963865, -3.233239256782939e-5, -2.9446543818443543e-5, -3.233239256661864e-5, 9.77631085099481e-5, 0.0001150782009903517, 0.00010930650350204183, 1.616619628486638e-5, -3.377531694146533e-5, -3.377531693979338e-5, 0.00017243942702166684, -4.3492822156587754e-5, 1.7609120659338027e-5, -3.233239256732964e-5, -1.943501043301523e-5, -1.6549161688085145e-5, 1.7609120656943624e-5, 1.7609120657164575e-5, -1.654916168326219e-5, 1.0599589726790801e-6, 0.0001064206547737045, 1.6166196284696962e-5, 0.00017243942701481235, -2.885848746713206e-6, -2.9446543823936345e-5, 1.616619628203475e-5, 0.00017243942701476744, 5.388732094192844e-6, -4.0989938812008666e-5, -1.4723271901231637e-5, 1.059958972525231e-6, 5.388732093570382e-6, -1.6166196284496453e-5, 0.000172439427016664, -4.34928221581365e-5, -3.377531694137636e-5

In [ ]:
# save decomposition
@save "./data/cnc_stab_decomposition_7.jld" x_value

In [ ]:
stab_1_decomposition = load("./data/stab_decomposition_1.jld")["x_value"]
stab1 = cncs_n_0[1][:,findall(x->x!=0,stab_1_decomposition)]

In [66]:
# 8-qubit cncs:
c8 = kron(c7,stab1)

65536×9288 Matrix{Rational{Int64}}:
  1  1  1   1  1  1   1  1  1   1  1  …  1  1   1  1  1   1  1  1   1  1  1
  0  1  0   0  1  0   0  1  0   0  1     1  0   0  1  0   0  1  0   0  1  0
 -1  0  1  -1  0  1  -1  0  1  -1  0     0  1  -1  0  1  -1  0  1  -1  0  1
  0  0  0   0  0  0   0  0  0   0  0     0  0   0  0  0   0  0  0   0  0  0
  0  0  0   0  0  0   0  0  0   0  0     0  0   0  0  0   0  0  0   1  1  1
  0  0  0   0  0  0   0  0  0   0  0  …  0  0   0  0  0   0  0  0   0  1  0
  0  0  0   0  0  0   0  0  0   0  0     0  0   0  0  0   0  0  0  -1  0  1
  0  0  0   0  0  0   0  0  0   0  0     0  0   0  0  0   0  0  0   0  0  0
  0  0  0   0  0  0   0  0  0   0  0     1  1   0  0  0   0  0  0   0  0  0
  0  0  0   0  0  0   0  0  0   0  0     1  0   0  0  0   0  0  0   0  0  0
  ⋮               ⋮                ⋮  ⋱            ⋮                ⋮     
  0  0  0   0  0  0   0  0  0   0  0     0  0   0  0  0   0  0  0   0  0  0
  0  0  0   0  0  0   0  0  0   0  0     0  0   0  0 

In [67]:
@save "./data/cnc_stab_8.jld" c8

In [70]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c8)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c8) * x .== T_states[8])

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value,"\n")

# save decomposition
@save "./data/cnc_stab_decomposition_8.jld" x_value

InterruptException: InterruptException:

In [75]:
c7

16384×3096 Matrix{Rational{Int64}}:
 1  1   1  1  1  1  1  1  1  1  1  1  …   1  1  1  1  1   1  1  1  1   1  1
 0  0   0  0  1  0  0  0  1  1  1  0      0  1  1  0  1   0  0  0  0   0  1
 0  0   0  0  0  0  0  1  0  0  0  1      0  0  0  1  0   0  1  1  0   0  0
 0  0   0  0  0  0  0  0  0  0  0  0      0  0  0  0  0   0  0  0  0   0  0
 1  0   0  1  0  1  1  0  1  0  0  0      0  1  0  1  0   0  0  0  1   0  0
 0  0   0  0  0  0  0  0  1  0  0  0  …   0  1  0  0  0  -1  0  0  0  -1  0
 0  0  -1  0  0  0  0  0  0  0  0  0     -1  0  0  1  0   0  0  0  0   0  0
 0  0   0  0  0  0  0  0  0  0  0  0      0  0  0  0  0   0  0  0  0   0  0
 0  1   0  0  1  0  0  0  0  1  1  0      0  0  0  0  0   0  1  1  0   0  0
 0  0  -1  0  1  0  0  0  0  1  1  0     -1  0  0  0  0   0  0  0  0   0  0
 ⋮               ⋮              ⋮     ⋱   ⋮               ⋮               ⋮
 0  0   0  0  0  0  0  0  0  0  0  0  …   0  0  0  0  0   0  0  0  0   0  0
 0  0   0  0  0  0  0  0  0  0  0  0      0  0  0  0

In [82]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c7[:,findall(x->abs(x)>10^-5,x_value_7)])[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c7[:,findall(x->abs(x)>10^-5,x_value_7)]) * x .== T_states[7])

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value,"\n")

# save decomposition
@save "./data/cnc_stab_decomposition_8.jld" x_value

Optimal objective value (min ||x||_1): 3.722187065517977
Optimal value of x: [7.117601239062846e-5, -2.5281911960909106e-5, -8.849455951434201e-5, 0.0002477853078597089, -6.642839033859108e-5, 2.5764232585152848e-5, 8.69081374649011e-6, 1.5542914036736454e-5, -7.036192203315215e-5, 9.668379288196655e-5, -2.5689170740163254e-5, 8.906122566649015e-5, -2.4612097518877014e-5, 7.701451010764915e-5, -0.0001510203556336711, -1.1548043679559866e-6, 4.8309334930213936e-6, -0.00010627983482205432, -3.5248482606032874e-5, -7.117601239079944e-5, 2.5281911960606983e-5, 1.4480006932526796e-5, 6.63164585408573e-5, 9.049165334301688e-5, 0.0002132653977070894, 8.90612256664857e-5, 0.00013612806160746838, 0.00032101894346762375, -0.00010075566752205255, -1.919397451233181e-5, 2.5689170740059634e-5, 0.0002424444829668197, -0.00010129186609520249, 2.576423258515935e-5, -5.7885936513723395e-5, 1.1548043680003478e-6, 1.9193974511914643e-5, 0.00022894053173115172, -7.814541460710756e-6, -5.7885936513858805e-

# cncs in optimal decomposition

In [113]:
# Open a file for writing
open("stab_states_in_decomposition_1234.txt", "w") do io
    for n in 1:4
        println(io, "Optimal decomposition for $(n) qubit(s)")
        decomp = load("./data/stab_decomposition_$(n).jld")["x_value"]
        nonzero = findall(x -> abs(x) > (10^(-6)), decomp)
        for x in nonzero
            sign = 0;
            if decomp[x]>0; sign = "+1"; else; sign = "-1"; end;
            println(io,"Has sign: $(sign)\n", pauli_basis_to_cnc(cncs_n_0[n][:,x], ps[n]))
        end
        println(io, "\n")
    end
end


In [115]:
# Open a file for writing
open("cncs_in_decomposition_123.txt", "w") do io
    for n in 1:3
        println(io, "Optimal decomposition for $(n) qubit(s)")
        decomp = load("./data/cnc_decomposition_$(n).jld")["x_value"]
        nonzero = findall(x -> abs(x) > (10^(-6)), decomp)
        for x in nonzero
            sign = 0;
            if decomp[x]>0; sign = "+1"; else; sign = "-1"; end;
            println(io,"Has sign: $(sign)\n", pauli_basis_to_cnc(cncs_n_1[n][:,x], ps[n]))
        end
        println(io, "\n")
    end
end


# simulated annealing

In [125]:
using Optim, LinearAlgebra

function objective(W, A, rho; penalty=1e6, trace_penalty=1e1, norm_type=1)
    # L1-norm of W (sparsity-inducing)
    L1_norm = sum(abs.(W))

    # Constraint violation (penalty for constraint mismatch)
    constraint_violation = norm(A * W .- rho, norm_type)

    # Ensure the sum of weights equals 1
    trace_violation = abs(sum(W) - 1)

    # Total cost
    return L1_norm + penalty * constraint_violation + trace_penalty * trace_violation
end


objective (generic function with 1 method)

In [106]:
using Optim

# Example data
A = load("./data/cnc_3_1.jld")["cnc_n_1"]  # (3,1)-cnc operators

# target quantum state:
rho = T_states[3];

# dimensions and size of problem:
d = length(rho); N = size(A)[2];

In [126]:
# Initial guess for W
W0 = vcat(1,rand(N-1))   # Centered around zero

# Solve using simulated annealing
result = optimize(
    W -> objective(W, A, rho), 
    W0, 
    SimulatedAnnealing(), 
    Optim.Options(iterations=30000)
);
Optim.minimum(result)

4.0751460530572453e9

# random subset of cnc vectors

In [10]:
# load 4-1 cnc vectors:
A4 = load("./large_data/cnc_vectors_4_1.jld")["cnc_vectors_4_1"]

256×731328 Matrix{Rational{Int64}}:
 1  1   1   1   1  1   1  1  1   1  1  1  …   1   1   1   1   1   1  1  1   1
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0      1  -1  -1  -1  -1  -1  1  1  -1
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0  …   0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 ⋮                 ⋮                ⋮     ⋱       ⋮                  ⋮     
 0  0   0   0   0  0   0  0  0   0  0  0      0   0   0   0   0   0  0  0   0
 1  1  -1  -1  -1  1  -1  1  1

In [11]:
using StatsBase

# Define number of variables:
n = 4; N = size(A4)[2]

# Define your collection
collection = 1:N

1:731328

In [ ]:
using JuMP
using GLPK

# Sample 5 unique elements without replacement
sampled_elements = sample(collection, 100000, replace=false);

# random submatrix
_A4 = A4[:,sampled_elements];

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(_A4)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(_A4) * x .== T_states[4])

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

In [92]:
cnc_decomposition_4_1 = (sampled_elements,x_value,obj_value)

([721207, 66574, 346098, 704507, 691244, 72597, 247117, 53568, 672440, 555870  …  129311, 595020, 58666, 98030, 558672, 344306, 342191, 697192, 363501, 510034], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 2.1717735386292367)

In [93]:
cnc_decomposition_4_1[3]

2.1717735386292367

In [94]:
@save "./data/cnc_decomposition_4_1.jld" cnc_decomposition_4_1